In [1]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler

In [2]:
cruise_data = spark.read.csv("/FileStore/tables/cruise_ship_info.csv", inferSchema=True, header = True)
cruise_data.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
 Ship_name|Cruise_line|Age| Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
 Journey| Azamara| 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55|
 Quest| Azamara| 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55|
Celebration| Carnival| 26| 47.262| 14.86| 7.22| 7.43| 31.8| 6.7|
 Conquest| Carnival| 11| 110.0| 29.74| 9.53| 14.88| 36.99|19.1|
 Destiny| Carnival| 17| 101.353| 26.42| 8.92| 13.21| 38.36|10.0|
 Ecstasy| Carnival| 22| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2|
 Elation| Carnival| 15| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2|
 Fantasy| Carnival| 23| 70.367| 20.56| 8.55| 10.22| 34.23| 9.2|
Fascination| Carnival| 19| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2|
 Freedom| Carnival| 6|110.23899999999999| 37.0| 9.51| 14.87| 29.79|11.5|
 Glory| Carnival| 10| 110.0| 29.74| 9.51| 14.87| 36.99|11.6|
 Holiday| Carnival| 28| 46.052| 14.52| 7.27| 7.26| 31.72| 6.6|
Imagination| Carnival| 18| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2|
Inspiration| Carnival| 17| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2|
 Legend| Carnival| 11| 86.0| 21.24| 9.63| 10.62| 40.49| 9.3|
 Liberty*| Carnival| 8| 110.0| 29.74| 9.51| 14.87| 36.99|11.6|
 Miracle| Carnival| 9| 88.5| 21.24| 9.63| 10.62| 41.67|10.3|
 Paradise| Carnival| 15| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2|
 Pride| Carnival| 12| 88.5| 21.24| 9.63| 11.62| 41.67| 9.3|
 Sensation| Carnival| 20| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
only showing top 20 rows

In [3]:
cruise_data.printSchema()

root
-- Ship_name: string (nullable = true)
-- Cruise_line: string (nullable = true)
-- Age: integer (nullable = true)
-- Tonnage: double (nullable = true)
-- passengers: double (nullable = true)
-- length: double (nullable = true)
-- cabins: double (nullable = true)
-- passenger_density: double (nullable = true)
-- crew: double (nullable = true)

In [4]:
#check if the predictor variables are highly correlated with the independent variable (crew)

from pyspark.sql.functions import corr

cruise_data.select(corr('Tonnage', 'crew')).show()
cruise_data.select(corr('Age', 'crew')).show()
cruise_data.select(corr('passengers', 'crew')).show()
cruise_data.select(corr('length', 'crew')).show()
cruise_data.select(corr('cabins', 'crew')).show()

+-------------------+
corr(Tonnage, crew)|
+-------------------+
 0.9275688115449388|
+-------------------+

+-------------------+
 corr(Age, crew)|
+-------------------+
-0.5306565039638852|
+-------------------+

+----------------------+
corr(passengers, crew)|
+----------------------+
 0.9152341306065384|
+----------------------+

+------------------+
corr(length, crew)|
+------------------+
 0.895856627101658|
+------------------+

+------------------+
corr(cabins, crew)|
+------------------+
0.9508226063578497|
+------------------+

In [5]:
#StringIndexer to convert categorical variable to numeric
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol='Cruise_line', outputCol='Cruise_line_index')
indexed_data = indexer.fit(cruise_data).transform(cruise_data)
indexed_data.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-----------------+
 Ship_name|Cruise_line|Age| Tonnage|passengers|length|cabins|passenger_density|crew|Cruise_line_index|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-----------------+
 Journey| Azamara| 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55| 16.0|
 Quest| Azamara| 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55| 16.0|
Celebration| Carnival| 26| 47.262| 14.86| 7.22| 7.43| 31.8| 6.7| 1.0|
 Conquest| Carnival| 11| 110.0| 29.74| 9.53| 14.88| 36.99|19.1| 1.0|
 Destiny| Carnival| 17| 101.353| 26.42| 8.92| 13.21| 38.36|10.0| 1.0|
 Ecstasy| Carnival| 22| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 Elation| Carnival| 15| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 Fantasy| Carnival| 23| 70.367| 20.56| 8.55| 10.22| 34.23| 9.2| 1.0|
Fascination| Carnival| 19| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 Freedom| Carnival| 6|110.23899999999999| 37.0| 9.51| 14.87| 29.79|11.5| 1.0|
 Glory| Carnival| 10| 110.0| 29.74| 9.51| 14.87| 36.99|11.6| 1.0|
 Holiday| Carnival| 28| 46.052| 14.52| 7.27| 7.26| 31.72| 6.6| 1.0|
Imagination| Carnival| 18| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
Inspiration| Carnival| 17| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 Legend| Carnival| 11| 86.0| 21.24| 9.63| 10.62| 40.49| 9.3| 1.0|
 Liberty*| Carnival| 8| 110.0| 29.74| 9.51| 14.87| 36.99|11.6| 1.0|
 Miracle| Carnival| 9| 88.5| 21.24| 9.63| 10.62| 41.67|10.3| 1.0|
 Paradise| Carnival| 15| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 Pride| Carnival| 12| 88.5| 21.24| 9.63| 11.62| 41.67| 9.3| 1.0|
 Sensation| Carnival| 20| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-----------------+
only showing top 20 rows

In [6]:
indexed_data.columns

Out[16]: ['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Cruise_line_index']

In [7]:
#To create additional column with all the features as a dense vector
assembler = VectorAssembler(inputCols=['Cruise_line_index', 'Age', 'Tonnage', 'passengers','length','cabins','passenger_density'], outputCol='features')
vectorized_data = assembler.transform(indexed_data)
vectorized_data.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-----------------+--------------------+
 Ship_name|Cruise_line|Age| Tonnage|passengers|length|cabins|passenger_density|crew|Cruise_line_index| features|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-----------------+--------------------+
 Journey| Azamara| 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55| 16.0|[16.0,6.0,30.2769...|
 Quest| Azamara| 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55| 16.0|[16.0,6.0,30.2769...|
Celebration| Carnival| 26| 47.262| 14.86| 7.22| 7.43| 31.8| 6.7| 1.0|[1.0,26.0,47.262,...|
 Conquest| Carnival| 11| 110.0| 29.74| 9.53| 14.88| 36.99|19.1| 1.0|[1.0,11.0,110.0,2...|
 Destiny| Carnival| 17| 101.353| 26.42| 8.92| 13.21| 38.36|10.0| 1.0|[1.0,17.0,101.353...|
 Ecstasy| Carnival| 22| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|[1.0,22.0,70.367,...|
 Elation| Carnival| 15| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|[1.0,15.0,70.367,...|
 Fantasy| Carnival| 23| 70.367| 20.56| 8.55| 10.22| 34.23| 9.2| 1.0|[1.0,23.0,70.367,...|
Fascination| Carnival| 19| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|[1.0,19.0,70.367,...|
 Freedom| Carnival| 6|110.23899999999999| 37.0| 9.51| 14.87| 29.79|11.5| 1.0|[1.0,6.0,110.2389...|
 Glory| Carnival| 10| 110.0| 29.74| 9.51| 14.87| 36.99|11.6| 1.0|[1.0,10.0,110.0,2...|
 Holiday| Carnival| 28| 46.052| 14.52| 7.27| 7.26| 31.72| 6.6| 1.0|[1.0,28.0,46.052,...|
Imagination| Carnival| 18| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|[1.0,18.0,70.367,...|
Inspiration| Carnival| 17| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|[1.0,17.0,70.367,...|
 Legend| Carnival| 11| 86.0| 21.24| 9.63| 10.62| 40.49| 9.3| 1.0|[1.0,11.0,86.0,21...|
 Liberty*| Carnival| 8| 110.0| 29.74| 9.51| 14.87| 36.99|11.6| 1.0|[1.0,8.0,110.0,29...|
 Miracle| Carnival| 9| 88.5| 21.24| 9.63| 10.62| 41.67|10.3| 1.0|[1.0,9.0,88.5,21....|
 Paradise| Carnival| 15| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|[1.0,15.0,70.367,...|
 Pride| Carnival| 12| 88.5| 21.24| 9.63| 11.62| 41.67| 9.3| 1.0|[1.0,12.0,88.5,21...|
 Sensation| Carnival| 20| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|[1.0,20.0,70.367,...|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-----------------+--------------------+
only showing top 20 rows

In [8]:
final_data = vectorized_data.select('features', 'crew')
final_data.show()

+--------------------+----+
 features|crew|
+--------------------+----+
[16.0,6.0,30.2769...|3.55|
[16.0,6.0,30.2769...|3.55|
[1.0,26.0,47.262,...| 6.7|
[1.0,11.0,110.0,2...|19.1|
[1.0,17.0,101.353...|10.0|
[1.0,22.0,70.367,...| 9.2|
[1.0,15.0,70.367,...| 9.2|
[1.0,23.0,70.367,...| 9.2|
[1.0,19.0,70.367,...| 9.2|
[1.0,6.0,110.2389...|11.5|
[1.0,10.0,110.0,2...|11.6|
[1.0,28.0,46.052,...| 6.6|
[1.0,18.0,70.367,...| 9.2|
[1.0,17.0,70.367,...| 9.2|
[1.0,11.0,86.0,21...| 9.3|
[1.0,8.0,110.0,29...|11.6|
[1.0,9.0,88.5,21....|10.3|
[1.0,15.0,70.367,...| 9.2|
[1.0,12.0,88.5,21...| 9.3|
[1.0,20.0,70.367,...| 9.2|
+--------------------+----+
only showing top 20 rows

In [9]:
#train and test split

train_data, test_data = final_data.randomSplit([0.7, 0.3])
print("Train data:", train_data.describe().show())
print("Test data:", test_data.describe().show())

+-------+------------------+
summary| crew|
+-------+------------------+
 count| 114|
 mean|7.7460526315789515|
 stddev|3.5609538392112037|
 min| 0.59|
 max| 21.0|
+-------+------------------+

Train data: None
+-------+------------------+
summary| crew|
+-------+------------------+
 count| 44|
 mean|7.9188636363636355|
 stddev|3.3868804442677356|
 min| 1.46|
 max| 13.6|
+-------+------------------+

Test data: None

In [10]:
lr = LinearRegression(labelCol='crew')
lr_model = lr.fit(train_data)
test_model = lr_model.evaluate(test_data)

In [11]:
test_model.rootMeanSquaredError

Out[28]: 0.7284694173252815

In [12]:
test_model.r2

Out[29]: 0.9526623018800269

In [13]:
unlabeled_data = test_data.select('features')
predictions = lr_model.transform(unlabeled_data)
predictions.show()

+--------------------+------------------+
 features| prediction|
+--------------------+------------------+
[0.0,5.0,160.0,36...|15.206231567308228|
[0.0,6.0,158.0,43...|14.091793028682366|
[0.0,7.0,158.0,43...|14.021663161128666|
[0.0,9.0,90.09,25...| 9.38978112182945|
[0.0,11.0,90.09,2...| 8.968308182156857|
[0.0,14.0,138.0,3...|12.985856554633218|
[0.0,15.0,78.491,...| 8.298151833492918|
[0.0,22.0,73.941,...| 9.120884467569729|
[1.0,12.0,88.5,21...| 9.548100427809683|
[1.0,17.0,70.367,...| 8.716482526261066|
[1.0,17.0,101.353...|10.856611430279322|
[1.0,18.0,70.367,...| 8.696986603196263|
[1.0,23.0,70.367,...| 8.611553812967658|
[2.0,7.0,116.0,31...|12.850115208238456|
[2.0,8.0,77.499,1...| 8.727525054702898|
[2.0,10.0,91.6270...| 9.256236702115137|
[2.0,11.0,108.977...| 11.17370835591745|
[3.0,13.0,63.0,14...| 6.754834398837652|
[3.0,16.0,59.652,...| 6.315684892643052|
[3.0,17.0,55.451,...| 5.885543437389586|
+--------------------+------------------+
only showing top 20 rows